In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from drone_sar.ipsar_dataset import IPSARDataset
from drone_sar.lightning_detector import get_lightning_trainer, LightningDetector
from collections import Counter

In [4]:
train_data = IPSARDataset("../heridal/trainImages/")
test_data = IPSARDataset("../heridal/testImages")
item = test_data[0]

In [5]:
trainer = get_lightning_trainer("IPSAR-DETR-FIXED", max_epochs=1000)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [123]:
# best_model_path = "/home/iz/workspace/DroneSearchAndRescue/notebooks/.checkpoints/epoch=88-step=14151.ckpt"
best_model_path = "/home/iz/workspace/DroneSearchAndRescue/notebooks/.checkpoints/epoch=65-step=10494.ckpt"

In [124]:
best_model = LightningDetector.load_from_checkpoint(best_model_path, lr=0.01)

/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLdq85T6-py3.10/lib/python3.10/site-packages/transformers/models/detr/image_processing_detr.py:776: FutureWarning: The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
  warnings.warn(
Some weights of DetrForObjectDetection were not initialized from the model checkpoint at facebook/detr-resnet-50 and are newly initialized because the shapes did not match:
- class_labels_classifier.weight: found shape torch.Size([92, 256]) in the checkpoint and torch.Size([2, 256]) in the model instantiated
- class_labels_classifier.bias: found shape torch.Size([92]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [125]:
test_dl_b1 = test_data.get_dataloader(best_model.processor, bs=1, shuffle=False)
train_dl_b1 = train_data.get_dataloader(best_model.processor, bs=1, shuffle=False)

In [126]:
best_val_outputs = trainer.predict(best_model, test_dl_b1)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/iz/.cache/pypoetry/virtualenvs/drone-sar-xLdq85T6-py3.10/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Predicting: |                                                      | 0/? [00:00<?, ?it/s]

In [128]:
best_train_outputs = trainer.predict(best_model, train_dl_b1)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |                                                      | 0/? [00:00<?, ?it/s]

In [129]:
# outputs = best_train_outputs
# target_ds = train_data

outputs = best_val_outputs
target_ds = test_data

In [130]:
import numpy as np

In [131]:
loss_sort_index = np.argsort([o["loss"] for o in outputs])
sorted_outputs = np.array(outputs)[loss_sort_index]
sorted_ds = np.array(list(target_ds))[loss_sort_index]

In [163]:
id = 90
target_output = sorted_outputs[id]
target_item = sorted_ds[id]
post_procesed_out = best_model.processor.post_process_object_detection(
    target_output,
    threshold=0.5,
    target_sizes=[target_item["target_sizes"][::-1]]
)[0]

X0, Y0, X1, Y1 = post_procesed_out["boxes"].T
W, H = X1 - X0, Y1 - Y0 
post_procesed_out["boxes"][:, 2] = W + 40
post_procesed_out["boxes"][:, 3] = H + 40

In [ ]:
from drone_sar.vis import plt_show

plt_show(
    target_item["pil"],
    im_opacity=0.6,
    colored_boxes_dict={"red": target_item["boxes"], "blue": post_procesed_out["boxes"]}
)